# CacheBackedEmbeddings - 임베딩 캐싱을 통한 성능 최적화

## 목차
1. [CacheBackedEmbeddings 개요](#1-cachebackedembeddings-개요)
2. [LocalFileStore를 사용한 영구 캐싱](#2-localfilestore를-사용한-영구-캐싱)
3. [InMemoryByteStore를 사용한 임시 캐싱](#3-inmemorybytestore를-사용한-임시-캐싱)

---

## 1. CacheBackedEmbeddings 개요

**CacheBackedEmbeddings**는 임베딩 계산 결과를 캐싱하여 재사용할 수 있게 해주는 LangChain의 효율성 향상 도구입니다.

### 캐싱이 필요한 이유

임베딩 생성 작업은 다음과 같은 문제점을 가지고 있습니다:

| 문제점 | 설명 | 영향 |
|--------|------|------|
| **높은 계산 비용** | OpenAI API 호출 시 토큰 단위 요금 발생 | 반복 작업 시 누적 비용 증가 |
| **긴 처리 시간** | 네트워크 통신 및 모델 추론 시간 소요 | 사용자 경험 저하 |
| **중복 연산** | 동일한 텍스트에 대한 반복적인 임베딩 생성 | 시스템 자원 낭비 |

### CacheBackedEmbeddings 동작 원리

CacheBackedEmbeddings는 다음 방식으로 동작합니다:

1. **해시 키 생성**: 입력 텍스트를 고유한 해시값으로 변환
2. **캐시 확인**: 해당 해시값이 저장소에 존재하는지 확인
3. **조건부 처리**: 
   - 캐시 존재 시: 저장된 임베딩 반환
   - 캐시 미존재 시: 새로운 임베딩 생성 후 저장

### 주요 구성 요소

#### from_bytes_store 메서드 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| `underlying_embeddings` | Embeddings | 실제 임베딩 생성을 담당하는 모델 객체 |
| `document_embedding_cache` | ByteStore | 임베딩 결과를 저장할 저장소 |
| `namespace` | str | 다른 임베딩 모델과의 캐시 충돌 방지용 구분자 |

### 저장소 유형 비교

| 저장소 타입 | 영속성 | 속도 | 메모리 사용 | 적용 사례 |
|-------------|--------|------|-------------|-----------|
| **LocalFileStore** | 영구 보존 | 보통 | 디스크 사용 | 프로덕션 환경, 장기 프로젝트 |
| **InMemoryByteStore** | 임시 저장 | 빠름 | RAM 사용 | 개발/테스트, 일회성 작업 |

### 중요 사항

**namespace 매개변수는 필수 설정 항목입니다.** 서로 다른 임베딩 모델(예: `text-embedding-3-small`, `text-embedding-ada-002`)을 사용할 때 캐시 충돌을 방지하기 위해 반드시 설정해야 합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 2. LocalFileStore를 사용한 영구 캐싱

**LocalFileStore**는 임베딩 결과를 로컬 파일 시스템에 영구적으로 저장하는 캐싱 방식입니다.

### LocalFileStore 특징

#### 장점

| 장점 | 설명 |
|------|------|
| **영구 보존** | 프로그램 종료 후에도 캐시 데이터 유지 |
| **비용 절약** | 한 번 계산된 임베딩의 무제한 재사용 |
| **안정성** | 디스크 기반 저장으로 데이터 손실 위험 최소화 |

#### 단점

| 단점 | 설명 |
|------|------|
| **저장 공간 사용** | 디스크 용량 점유 |
| **관리 오버헤드** | 오래된 캐시 파일 정리 필요 |
| **I/O 지연** | 파일 시스템 접근으로 인한 약간의 지연 |

### 실제 구현 과정

다음 예제에서는 LocalFileStore를 사용하여 임베딩을 캐싱하고 FAISS 벡터 스토어와 연동하는 방법을 보여줍니다:

1. **OpenAI 임베딩 모델 설정**
2. **로컬 파일 저장소 생성**
3. **캐시 백엔드 임베딩 객체 생성**
4. **문서 처리 및 벡터 저장소 구성**

In [ ]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 파일 저장소 설정 - "./cache/" 폴더에 캐시 파일 저장
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,  # 실제 임베딩을 수행할 모델
    document_embedding_cache=store,  # 캐시를 저장할 저장소
    namespace=embedding.model,  # 모델별로 캐시를 구분하기 위한 네임스페이스
)

In [ ]:
# 현재 캐시 저장소에 저장된 키들을 확인 (아직 아무것도 없음)
list(store.yield_keys())

### 문서 처리 단계별 과정

임베딩 캐싱이 적용된 문서 처리는 다음과 같은 단계로 진행됩니다:

| 단계 | 작업 내용 | 도구/라이브러리 |
|------|-----------|----------------|
| **1. 문서 로드** | 텍스트 파일에서 원본 데이터 읽기 | TextLoader |
| **2. 텍스트 분할** | 긴 문서를 적절한 크기의 청크로 분할 | CharacterTextSplitter |
| **3. 임베딩 생성** | 각 청크를 벡터로 변환 (캐싱 적용) | CacheBackedEmbeddings |
| **4. 벡터 저장** | 생성된 임베딩을 검색 가능한 형태로 저장 | FAISS |

### 성능 최적화 효과

캐싱 시스템이 적용된 후의 성능 개선 효과는 다음과 같습니다:

| 실행 차수 | 처리 방식 | 예상 소요 시간 |
|-----------|-----------|----------------|
| **첫 번째** | API 호출 + 임베딩 계산 + 캐시 저장 | 수 초 |
| **두 번째 이후** | 캐시에서 직접 로드 | 수 밀리초 |

In [ ]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 텍스트 파일에서 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정 (1000자씩 나누고, 겹치는 부분은 0자)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 로드된 문서를 설정된 크기로 분할
documents = text_splitter.split_documents(raw_documents)

In [ ]:
# 첫 번째 실행 - 모든 문서를 임베딩하고 FAISS 데이터베이스 생성 (시간 측정)
%time db = FAISS.from_documents(documents, cached_embedder)

### 캐싱 성능 검증

동일한 문서셋에 대해 벡터 저장소를 재생성할 때의 성능 차이를 측정합니다.

#### 성능 비교 결과 분석

캐싱 메커니즘을 통한 성능 향상은 다음과 같이 측정됩니다:

| 측정 항목 | 첫 번째 실행 | 두 번째 실행 | 개선율 |
|-----------|--------------|--------------|--------|
| **API 호출 수** | 전체 청크 수만큼 | 0회 | 100% 감소 |
| **처리 시간** | 기준값 | 95% 이상 단축 | 95%+ 향상 |
| **비용** | 전체 토큰 요금 | 0원 | 100% 절약 |

이러한 성능 개선은 특히 반복적인 개발 작업이나 프로덕션 환경에서 큰 효과를 발휘합니다.

In [ ]:
# 두 번째 실행 - 캐싱된 임베딩을 사용하여 훨씬 빠르게 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)